In [ ]:
from jupyter_dash import JupyterDash     # pip install dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Output, Input

from dash_extensions import Lottie       # pip install dash-extensions
import dash_bootstrap_components as dbc  # pip install dash-bootstrap-components
import plotly.express as px              # pip install plotly
import pandas as pd                      # pip install pandas
from datetime import date
import calendar

In [ ]:
api = irl.Api( #leila-bert-group-recs
# find-tags
)
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.LUX])

In [ ]:
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardImg(src='/assets/irl-logo2.png')
            ],className='mb-2'),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.DatePickerSingle(
                        id='my-date-picker-start',
                        date=date(2018, 1, 1),
                        className='ml-5'
                    ),
                    dcc.DatePickerSingle(
                        id='my-date-picker-end',
                        date=date(2021, 4, 4),
                        className='mb-2 ml-2'
                    ),
                ])
            ], color="info", style={'height':'18vh'}),
        ], width=8),
    ],className='mb-2 mt-2'),
    dbc.Row([
        dbc.Col([
            dbc.Card([ #  users interested in relevant invites
                dbc.CardHeader(Lottie(options=options, width="67%", height="67%", url=url_coonections)),
                dbc.CardBody([
                    html.H6('Interested'),
                    html.H2(id='content-interested', children="000")
                ], style={'textAlign':'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([ #  invites attended in relevant invites
                dbc.CardHeader(Lottie(options=options, width="32%", height="32%", url=url_companies)),
                dbc.CardBody([
                    html.H6('Attended'),
                    html.H2(id='content-attended', children="000")
                ], style={'textAlign':'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([ # relevant groups joined
                dbc.CardHeader(Lottie(options=options, width="25%", height="25%", url=url_msg_in)),
                dbc.CardBody([
                    html.H6('Groups Joined'),
                    html.H2(id='content-gjoin', children="000")
                ], style={'textAlign':'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([ # relevant invites sent
                dbc.CardHeader(Lottie(options=options, width="53%", height="53%", url=url_msg_out)),
                dbc.CardBody([
                    html.H6('Invites sent'),
                    html.H2(id='content-sent', children="000")
                ], style={'textAlign': 'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([ # top groups
                dbc.CardHeader(Lottie(options=options, width="53%", height="53%", url=url_msg_out)),
                dbc.CardBody([
                    html.H6('Top Groups'),
                    html.H2(id='content-tags', children="000")
                ], style={'textAlign': 'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([ # top tags
                dbc.CardHeader(Lottie(options=options, width="53%", height="53%", url=url_msg_out)),
                dbc.CardBody([
                    html.H6('Top Tags'),
                    html.H2(id='content-tags', children="000")
                ], style={'textAlign': 'center'})
            ]),
        ], width=2),
    ],className='mb-2'),
    dbc.Row([
         dbc.Col([
            dbc.Card([
                dbc.CardBody([ # demographic age bar chart
                    dcc.Graph(id='bar-chart', figure={}, config={'displayModeBar': False}),
                ])
            ]),
        ], width=4),
    ],className='mb-2'),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([ # demographic gender pie chart
                    dcc.Graph(id='pie-chart', figure={}, config={'displayModeBar': False}),
                ])
            ]),
        ], width=3),
], fluid=True)


# Updating the 4 number cards ******************************************
@app.callback(
    Output('content-tags', 'children')
    Output('content-topgroups', 'children')
    Output('content-sent','children'),
    Output('content-gjoined','children'),
    Output('content-attended','children'),
    Output('content-interested','children'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date'),
)

# df_i = df_i.apply(find_tags) - invites sent, interested, attended : invites inner join invites_users on invite id 
# df_g = df_groups.apply(find_tags) - users groups 

def update_values(start_date, end_date):
    # invites attended, interested, sent
    df_i = df_i.copy()

    df_i = df_i[(df_i['start_date']>=start_date) & (df_i['start_date']<=end_date)]
    count_int = df_i['interested']
    int_num = len(count_int['user_id'].unique())
    
    count_att = df_i['attending']
    att_num = len(count_att['user_id'].unique())
    
    inv_num = len(df_i['inviter_id'].unique())
    
    
    df_g = df_g.copy()
    # groups joined
    df_g = df_g.copy()
    df_gj = df_g[(df_g['joined_at']>=start_date) & (df_g['joined_at']<=end_date)]
    join_num = len(df_gj)
    
    
    # top groups
    # apply bert endpoint to recommend top groups (3?)
    
    # top tags
    # apply find tags to those top groups
    

    return int_num, att_num, inv_num, join_num

# Pie Chart ************************************************************
@app.callback(
    Output('pie-chart','figure'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date'),
)

# df_dg - gender -> rc.query('SELECT gender FROM users WHERE gender IS NOT NULL')
def update_pie(start_date, end_date):
    df_dg = df_dg.copy()
    df_dg = df_dg[(df_dg['signup_date']>=start_date) & (df_dg['signup_date']<=end_date)]
    num_m = len(df_dg[df_dg['gender']=='m']) # males 
    num_f = len(df_dg[df_dg['gender'] == 'f']) # females
    fig_pie = px.pie(names=['Male','Female'], values=[num_m, num_f],
                     template='ggplot2', title="User Demographic - Gender"
                     )
    fig_pie.update_layout(margin=dict(l=20, r=20, t=30, b=20))
    fig_pie.update_traces(marker_colors=['red','blue'])

    return fig_pie

# Bar Chart ************************************************************
@app.callback(
    Output('bar-chart','figure'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date'),
)

# df_da - age -> rc.query('SELECT birthday FROM users WHERE birthday IS NOT NULL')

def update_bar(start_date, end_date):
    df_da = df_da.copy()
    df_da = df_da[(df_da['signup_date']>=start_date) & (df_da['signup_date']<=end_date)]

    birthdate = df_da['birthday']
    current = start_date
    diff = current.year - birthdate.year
    df_da['age'] = diff
    
    df_da = df_da[["age"]].value_counts().head(10)
    df_da = df_da.to_frame()
    df_da.reset_index(inplace=True)
    df_da.rename(columns={0:'Age'}, inplace=True)
    # print(dff_comp)
    fig_bar = px.bar(df_da, x='Age', y='Segments', template='ggplot2',
                      orientation='h', title="User Demographic - Age")
    fig_bar.update_yaxes(tickangle=45)
    fig_bar.update_layout(margin=dict(l=20, r=20, t=30, b=20))
    fig_bar.update_traces(marker_color='blue')

    return fig_bar